# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why? 

This is a classification problem, because the output we are predict is discrete rather than continuous. 

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from IPython.display import Image

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data['passed'][student_data['passed'] == 'yes' ].shape[0]
n_failed = student_data['passed'][student_data['passed'] == 'no' ].shape[0]
grad_rate = 100. * n_passed/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 1. * num_test/num_all, random_state=0)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.009


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.869198312236


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.758620689655


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

training_set_sizes = [100, 200, 300]

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
# TODO: Train and predict using two other models

In [11]:
# TODO: Train and predict using two other models
def train_predict_table(clf, training_set_sizes, X_train, y_train, X_test, y_test):
    columns = ["Size", "T-time", 
               "P-time(Train)", "Score(Train)", 
               "P-time(Test)", "Score(Test)"]
    
    table = pd.DataFrame(columns = columns)
    index = 0; 
    for size in training_set_sizes:
        X_train_subset = X_train.head(size)
        y_train_subset = y_train.head(size)                    

        start = time.time()
        clf.fit(X_train_subset, y_train_subset)
        end = time.time()
        training_time = end - start             

        start = time.time()
        y_pred = clf.predict(X_train_subset)
        end = time.time()
        prediction_time1 = end - start 
        training_score = f1_score(y_train_subset, y_pred, pos_label='yes')

        start = time.time()
        y_pred = clf.predict(X_test)
        end = time.time()
        prediction_time2 = end - start 
        testing_score = f1_score(y_test, y_pred, pos_label='yes')

        row = [size, training_time, 
               prediction_time1, training_score, 
               prediction_time2, testing_score]

        table.loc[index] = row
        index += 1
    print table
    print table.apply(np.mean, axis = 0)

## Support Vector Machines

*What are the general applications of this model?  What are its strengths and weaknesses?*

Strengths
- Work well in complicated domains with a clear margin of seperation
- Data does not need to be linearly separated due to the kernel transformations and several functional forms
- Effective in high dimensional spaces.-
- Still effective in cases where number of dimensions is greater than the number of samples.

Weakness
- Very slow
- Do not work well with large datasets
- Might overfit to the noise in the data
- Results depend on model selection 

*Given what you know about the data so far, why did you choose this model to apply?*

The data set was not too large for a SVC to handle.  

*Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.*

In [12]:
train_predict_table(SVC(), training_set_sizes, X_train, y_train, X_test, y_test)

    Size  T-time  P-time(Train)  Score(Train)  P-time(Test)  Score(Test)
0  100.0   0.001          0.001      0.859060         0.001     0.783784
1  200.0   0.005          0.003      0.869281         0.001     0.775510
2  300.0   0.009          0.006      0.869198         0.002     0.758621
Size             200.000000
T-time             0.005000
P-time(Train)      0.003333
Score(Train)       0.865847
P-time(Test)       0.001333
Score(Test)        0.772638
dtype: float64


## Decision Trees

*What are the general applications of this model?  What are its strengths and weaknesses?*

Strengths
- Very fast 
- Requires very little data preparation  
- Can be used for feature selection
- Used to build bigger classifers such as random forest 
- Easy to interpret and explain results

Weakness
- Prone to overfitting
- Works with poorly with indepedent features
- Need to monitor complexity and prevent the tree from growing too complex

*Given what you know about the data so far, why did you choose this model to apply?*

I knew that I'm going to have to explain this model to non-technical and the results of a decision are easier to explain.

*Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.*

In [13]:
train_predict_table(DecisionTreeClassifier(), training_set_sizes, X_train, y_train, X_test, y_test)

    Size  T-time  P-time(Train)  Score(Train)  P-time(Test)  Score(Test)
0  100.0   0.001          0.000           1.0           0.0     0.706897
1  200.0   0.002          0.000           1.0           0.0     0.713178
2  300.0   0.002          0.001           1.0           0.0     0.747967
Size             200.000000
T-time             0.001667
P-time(Train)      0.000333
Score(Train)       1.000000
P-time(Test)       0.000000
Score(Test)        0.722681
dtype: float64


## Random Forest Classifier

*What are the general applications of this model?  What are its strengths and weaknesses?*

Strengths (compared to decision trees)
- higher predicitve power 
- Less prone to overfitting

Weakness
- Very slow
- Need to train many decision trees

*Given what you know about the data so far, why did you choose this model to apply?*

I had already tried using decision trees.  I wanted to see if an ensemble method that utilized many decision trees would fare better than a single decision tree.  

*Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.*

In [14]:
train_predict_table(RandomForestClassifier(), training_set_sizes, X_train, y_train, X_test, y_test)

    Size  T-time  P-time(Train)  Score(Train)  P-time(Test)  Score(Test)
0  100.0   0.023          0.001      0.992248         0.001     0.766917
1  200.0   0.019          0.001      0.981273         0.001     0.797101
2  300.0   0.041          0.000      0.990338         0.000     0.753846
Size             200.000000
T-time             0.027667
P-time(Train)      0.000667
Score(Train)       0.987953
P-time(Test)       0.000667
Score(Test)        0.772622
dtype: float64


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

**Answer:**

*Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

Support vector machine had the best average F1 score in the testing set.  Random forest barely outperformed decision trees.  Decision trees were the fastest to train.  Random Forest was the slowest, but it scale better with more data than support vector machines.  The training time for decision trees and random tree increased much slower than support vector machines when the training set grew.  This means that support vector machines cannot be implemented on a larger scale whereas decisions trees and random forest can.  I decided to go with support vector machines since it was the most accurate, and the data set is small enough so computional resources aren't an issue.      

*In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).*

Imagine our students as balls on a table.  The blue balls represent the student that passed the class.  The red balls represent the students that did not passed.  The features of student represent the position on the table we place the ball.

<img src="balls.png" height="100" width="300" align='center'>

We want to create a boundary to separate the red balls from the blue balls.  There are many ways we can separate the balls.  Support vector machines tries to find a line of separation that maximizes the margin or gap from each side of the line.    

<img src="margin.png" height="100" width="300" align='center'>

Sometimes we simply cannot separate the balls with just a straight line.  In those, situations, support vector machines need to use a trick called the kernel transformation.  The gist of the trick can be thought up as throwing the balls in air and then use a sheet of paper to separate them.  The end result is the boundary will be a curvy line instead of a straight line.           

<img src="kernel.png" height="200" width="300" align='center'>

Suppose now that you have a new student.  You can place that new student on the table to see which boundary that student will fall into.  The support vector machine will predict that the new student will pass if that student ends up in the boundary with the blue balls.  Likewise, it will predict the new student will fail if that student ends up on the graph with the rell balls.          
    

*Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.*

In [15]:
from sklearn import grid_search
from sklearn.metrics import make_scorer

parameters = {'kernel':('linear', 'rbf','sigmoid'), 'C':[1, 10]} 
f1_scorer = make_scorer(f1_score, pos_label="yes")
clf = grid_search.GridSearchCV(SVC(), parameters, scoring=f1_scorer) 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
best_f1_score =  f1_score(y_test, y_pred, pos_label='yes')
best_clf = clf.best_estimator_
print best_clf

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


What is the model's final F<sub>1</sub> score?

In [16]:
print "Best F1 Score: {} ".format(best_f1_score)

Best F1 Score: 0.758620689655 
